# Parsing data down to usable data

#### In this notebook, we will take the output and perform a series of removals based on the output of biopsyAnnotation
1. Create a template for investigating data
2. Investigate all data, remove those that may be repeated (look for spaces in the biopsy name) [1stParse]
3. Use template to look at all data 
4. Remove biopsies w/ missing tissue or consent [2ndParse]
5. Remove those w/ no FFPE tissue [3rdParse]
6. Remove those w/ missing screenshots from BrainLab Neuronav software [4thParse]
7. Remove those w/o pathological outcome assigned in multnom_out (missing pathology) [5thParse]
8. Remove those whose imaging failed to be quantified for some reason [6thParse]
9. Remove ependymomas and other extraneous histologies [7thParse]
10. Remove necrotic samples [8thParse]
11. Only include rHGG or TxE [9thParse]

## 1. Create template: 

In [1]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

temp = matrix(nrow = 3, ncol = 4)
colnames(temp) = c("data", "patients", "scans", "samples")
temp[1,1] = "old_po1"
temp[2,1] = "REC_HGG"
temp[3,1] = "TOTAL"

## 2. Investigate data, remove those that look repeated, look for those w/ spaces in biopsy names (when merged w/ igt_stats data) 

In [2]:
recgli = read.csv("../AnnotateData/10thAnnot_researchPath_withInVivo.csv")

In [3]:
dim(recgli)

[1] 627  75

In [4]:
## looking at duplicates: 
table(duplicated(recgli$roi.label))
recgli[duplicated(recgli$roi.label),]


FALSE  TRUE 
  615    12 

,b_number,t_number,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,roi.label,tumor_cell_evaluation,necrosis,f8_delicate,f8_simple,f8_complex,mib_1,bx_pure_treatment_effect,nfse,nfl,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,phn_npar,recovn_npar,cni,ccri,crni,ncho,ncre,nnaa,laclip,nlip,nlac,olddata,newdata,comments,notes.,imaging_code,perf_quant,cbv_nlin,spec_quant,include_anat,include_diffu1000,include_diffu2000,include_diffu_all,include_perf,include_spec,include_one_advanced_mod,include_all_advanced_mods,X.CEL,X.NEL,X.NEC,sum,in_CEL,in_T2all,in_NEL,in_NEC,in_ROI,desired_hist,multnom_out,no_ffpe,waiting_on_path,rhgg_txe_analysis
381,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.70,0.50,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,,NA,NA,,NA,1,1,1,1,1,1,0,1,0,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1
382,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.70,0.50,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,3.51,2.14,0.94,0.69,0.26,0.07,0.40,0.12,0.28,0,1,,NA,NA,,NA,0,1,1,1,1,1,1,1,1,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1
383,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.70,0.50,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,3.51,2.14,0.94,0.69,0.26,0.07,0.40,0.12,0.28,0,1,,NA,NA,,NA,0,1,1,1,1,1,1,1,1,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1
390,3670,9288,10199,2014-12-16,2014-12-17,Grade IV,Glioblastoma,49B75,2,0,,,,5.92,,1.74,1.82,1.35,0.92,-3.35,1.64,0.72,1.26,1.55,1.83,1.54,1.71,0.72,NA,NA,NA,NA,1.09,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,"""foci of macrophages""",NA,NA,noPerf_or_makePerfIssue,NA,1,1,1,1,1,0,0,1,0,58.33,41.67,0,100,1,1,0,0,1,1,rHGG,0,0,1
391,3670,9288,10199,2014-12-16,2014-12-17,Grade IV,Glioblastoma,49B75,2,0,,,,5.92,,1.74,1.82,1.35,0.92,-3.35,1.64,0.72,1.26,1.55,1.83,1.54,1.71,0.72,NA,NA,NA,NA,1.09,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,"""foci of macrophages""",NA,NA,noPerf_or_makePerfIssue,NA,1,1,1,1,1,0,0,1,0,58.33,41.67,0,100,1,1,0,0,1,1,rHGG,0,0,1
392,3670,9288,10199,2014-12-16,2014-12-17,Grade IV,Glioblastoma,49B75,2,0,,,,5.92,,1.74,1.82,1.35,0.92,-3.35,1.64,0.72,1.26,1.55,1.83,1.54,1.71,0.72,NA,NA,NA,NA,1.09,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,"""foci of macrophages""",NA,NA,noPerf_or_makePerfIssue,NA,1,1,1,1,1,0,0,1,0,58.33,41.67,0,100,1,1,0,0,1,1,rHGG,0,0,1
425,3791,9970,10507,2015-08-25,2015-08-26,Treatment Effect,Treatment Effect,49B78,3,0,,,,10.53,f,2.11,1.83,0.96,0.68,-2.03,1.97,0.50,1.54,1.79,2.08,1.85,1.99,0.61,1.49,1.72,2.03,1.79,0.59,0.57,0.55,100,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,,NA,NA,,NA,1,1,1,1,1,1,0,1,0,20.00,80.00,0,100,0,1,1,0,1,1,rHGG,0,0,1
426,3791,9970,10507,2015-08-25,2015-08-26,Treatment Effect,Treatment Effect,49B78,3,0,,,,10.53,f,2.11,1.83,0.96,0.68,-2.03,1.97,0.50,1.54,1.79,2.08,1.85,1.99,0.61,1.49,1.72,2.03,1.79,0.59,0.57,0.55,100,1.54,0.51,1.02,0.68,0.51,0.27,0.86,0.86,0.86,0,1,,NA,NA,,NA,0,1,1,1,1,1,1,1,1,20.00,80.00,0,100,0,1,1,0,1,1,rHGG,0,0,1
427,3791,9970,10507,2015-08-25,2015-08-26,Treatment Effect,Treatment Effect,49B78,3,0,,,,10.53,f,2.11,1.83,0.96,0.68,-2.03,1.97,0.50,1.54,1.79,2.08,1.85,1.99,0.61,1.49,1.72,2.03,1.79,0.59,0.57,0.55,100,1.54,0.51,1.02,0.68,0.51,0.27,0.86,0.86,0.86,0,1,,NA,NA,,NA,0,1,1,1,1,1,1,1,1,20.00,80.00,0,100,0,1,1,0,1,1,rHGG,0,0,1
530,3783,11192,11150,2017-01-29,2017-01-30,Grade III,Astrocytoma,36B97,3,0,2,1,0,9.56,f,1.80,1.86,1.39,1.26,-1.17,1.67,0.73,1.42,1.60,1.77,1.60,1.69,1.06,1.40,1.46,1.64,1.52,0.67,0.79,0.75,100,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,,NA,NA,,NA,1,1,1,1,1,1,0,1,0,33.33,66.67,NA,100,1,1,1,0,1,1,rHGG,0,0,1


In [5]:
## looks like they all somehow got duplicated ... all have the exact same parameters, so we can just eliminate these. Just to make sure: 
recgli[grep('49B74', recgli$roi.label),]
## yep, all the same; we can actually now just delete these: 
recgli = recgli[!duplicated(recgli$roi.label),]
dim(recgli)

,b_number,t_number,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,roi.label,tumor_cell_evaluation,necrosis,f8_delicate,f8_simple,f8_complex,mib_1,bx_pure_treatment_effect,nfse,nfl,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,phn_npar,recovn_npar,cni,ccri,crni,ncho,ncre,nnaa,laclip,nlip,nlac,olddata,newdata,comments,notes.,imaging_code,perf_quant,cbv_nlin,spec_quant,include_anat,include_diffu1000,include_diffu2000,include_diffu_all,include_perf,include_spec,include_one_advanced_mod,include_all_advanced_mods,X.CEL,X.NEL,X.NEC,sum,in_CEL,in_T2all,in_NEL,in_NEC,in_ROI,desired_hist,multnom_out,no_ffpe,waiting_on_path,rhgg_txe_analysis
380,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.7,0.5,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,,NA,NA,,NA,1,1,1,1,1,1,0,1,0,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1
381,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.7,0.5,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,1,,NA,NA,,NA,1,1,1,1,1,1,0,1,0,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1
382,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.7,0.5,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,3.51,2.14,0.94,0.69,0.26,0.07,0.4,0.12,0.28,0,1,,NA,NA,,NA,0,1,1,1,1,1,1,1,1,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1
383,3645,9213,10138,2014-10-22,2014-10-23,Grade IV,Glioblastoma,49B74,3,0,,,,40.81,f,1.65,1.55,1.32,0.66,-6.92,1.7,0.5,1.39,1.66,1.83,1.73,1.74,0.61,NA,NA,NA,NA,1.62,1.29,1.24,100,3.51,2.14,0.94,0.69,0.26,0.07,0.4,0.12,0.28,0,1,,NA,NA,,NA,0,1,1,1,1,1,1,1,1,95.74,4.26,NA,100,1,1,0,0,1,1,rHGG,0,0,1


[1] 615  75

In [6]:
## looking at spaces: 
recgli[grep(' ', recgli$roi.label),]

b_number,t_number,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,roi.label,tumor_cell_evaluation,necrosis,f8_delicate,f8_simple,f8_complex,mib_1,bx_pure_treatment_effect,nfse,nfl,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,phn_npar,recovn_npar,cni,ccri,crni,ncho,ncre,nnaa,laclip,nlip,nlac,olddata,newdata,comments,notes.,imaging_code,perf_quant,cbv_nlin,spec_quant,include_anat,include_diffu1000,include_diffu2000,include_diffu_all,include_perf,include_spec,include_one_advanced_mod,include_all_advanced_mods,X.CEL,X.NEL,X.NEC,sum,in_CEL,in_T2all,in_NEL,in_NEC,in_ROI,desired_hist,multnom_out,no_ffpe,waiting_on_path,rhgg_txe_analysis


In [7]:
## if we delete the spaces, do we have duplicates ? or no? 
recgli$roi.label = gsub(" ", "", recgli$roi.label)

In [8]:
table(duplicated(recgli$roi.label))


FALSE 
  615 

In [9]:
## great, so the spaces don't really matter all that much it turns out. Now we have 615 biopsies to start with. 

In [10]:
write.csv(recgli, "1stParse_researchpath_withInVivo.csv", row.names = F)

## 3. Use template to look at all data


In [11]:
recgli = read.csv("1stParse_researchpath_withInVivo.csv")
dim(recgli)

[1] 615  75

In [12]:
all_possible = temp
all_possible[1,2] = sum(ifelse(duplicated(recgli$b_number)==FALSE & recgli$olddata==1, 1, 0))
all_possible[1,3] = sum(ifelse(duplicated(recgli$t_number)==FALSE & recgli$olddata==1, 1, 0))
all_possible[1,4] = sum(recgli$olddata==1)
all_possible[2,2] = sum(ifelse(duplicated(recgli$b_number)==FALSE & recgli$newdata==1, 1, 0))
all_possible[2,3] = sum(ifelse(duplicated(recgli$t_number)==FALSE & recgli$newdata==1, 1, 0))
all_possible[2,4] = sum(recgli$newdata==1)
all_possible[3,2:4]= as.numeric(all_possible[1,2:4]) + as.numeric(all_possible[2,2:4])
all_possible

data,patients,scans,samples
old_po1,124,129,316
REC_HGG,91,95,299
TOTAL,215,224,615


## 4. Remove samples without tissue/consent: 

In [13]:
recgli.noconsent = recgli[-which(recgli$imaging_code=="tiss_cons"),]
dim(recgli.noconsent)
noconsent = temp
noconsent[1,2] = sum(ifelse(duplicated(recgli.noconsent$b_number)==FALSE & recgli.noconsent$olddata==1, 1, 0))
noconsent[1,3] = sum(ifelse(duplicated(recgli.noconsent$t_number)==FALSE & recgli.noconsent$olddata==1, 1, 0))
noconsent[1,4] = sum(recgli.noconsent$olddata==1)
noconsent[2,2] = sum(ifelse(duplicated(recgli.noconsent$b_number)==FALSE & recgli.noconsent$newdata==1, 1, 0))
noconsent[2,3] = sum(ifelse(duplicated(recgli.noconsent$t_number)==FALSE & recgli.noconsent$newdata==1, 1, 0))
noconsent[2,4] = sum(recgli.noconsent$newdata==1)
noconsent[3,2:4]= as.numeric(noconsent[1,2:4]) + as.numeric(noconsent[2,2:4])
noconsent

[1] 607  75

data,patients,scans,samples
old_po1,123,128,313
REC_HGG,89,93,294
TOTAL,212,221,607


In [14]:
write.csv(recgli.noconsent, "2ndparse_researchPath_withInVivo.csv", row.names = F)

## 5. Remove those w/ no FFPE tissue

In [15]:
recgli.noconsent = read.csv("2ndparse_researchPath_withInVivo.csv")

In [16]:
recgli.noffpe = recgli.noconsent[-which(recgli.noconsent$no_ffpe==1),]
dim(recgli.noffpe) #584
noffpe = temp
noffpe[1,2] = sum(ifelse(duplicated(recgli.noffpe$b_number)==FALSE & recgli.noffpe$olddata==1, 1, 0))
noffpe[1,3] = sum(ifelse(duplicated(recgli.noffpe$t_number)==FALSE & recgli.noffpe$olddata==1, 1, 0))
noffpe[1,4] = sum(recgli.noffpe$olddata==1)
noffpe[2,2] = sum(ifelse(duplicated(recgli.noffpe$b_number)==FALSE & recgli.noffpe$newdata==1, 1, 0))
noffpe[2,3] = sum(ifelse(duplicated(recgli.noffpe$t_number)==FALSE & recgli.noffpe$newdata==1, 1, 0))
noffpe[2,4] = sum(recgli.noffpe$newdata==1)
noffpe[3,2:4]= as.numeric(noffpe[1,2:4]) + as.numeric(noffpe[2,2:4])
noffpe 

[1] 585  75

data,patients,scans,samples
old_po1,123,128,313
REC_HGG,89,92,272
TOTAL,212,220,585


In [17]:
write.csv(recgli.noffpe, "3rdParse_researchPath_withInVivo.csv", row.names = F)

## 6. Remove those w/ missing screenshots from BrainLab Neuronav software


In [18]:
recgli.noffpe = read.csv("3rdParse_researchPath_withInVivo.csv")
dim(recgli.noffpe)

[1] 585  75

In [19]:
recgli.noscreenshots = recgli.noffpe[-which(recgli.noffpe$imaging_code=="scrn"),]
dim(recgli.noscreenshots)  # 541 
noscreenshots = temp
noscreenshots[1,2] = sum(ifelse(duplicated(recgli.noscreenshots$b_number)==FALSE & recgli.noscreenshots$olddata==1, 1, 0))
noscreenshots[1,3] = sum(ifelse(duplicated(recgli.noscreenshots$t_number)==FALSE & recgli.noscreenshots$olddata==1, 1, 0))
noscreenshots[1,4] = sum(recgli.noscreenshots$olddata==1)
noscreenshots[2,2] = sum(ifelse(duplicated(recgli.noscreenshots$b_number)==FALSE & recgli.noscreenshots$newdata==1, 1, 0))
noscreenshots[2,3] = sum(ifelse(duplicated(recgli.noscreenshots$t_number)==FALSE & recgli.noscreenshots$newdata==1, 1, 0))
noscreenshots[2,4] = sum(recgli.noscreenshots$newdata==1)
noscreenshots[3,2:4]= as.numeric(noscreenshots[1,2:4]) + as.numeric(noscreenshots[2,2:4])
noscreenshots

[1] 536  75

data,patients,scans,samples
old_po1,110,113,275
REC_HGG,86,89,261
TOTAL,196,202,536


In [20]:
write.csv(recgli.noscreenshots, "4thParse_researchPath_withInVivo.csv", row.names = F)

## 7. Remove those w/o pathological outcome assigned in multnom_out (missing pathology)


In [21]:
recgli.noscreenshots = read.csv("4thParse_researchPath_withInVivo.csv")

In [22]:
nopath_index = c(grep("dneDNI", recgli.noscreenshots$multnom_out))
recgli.nopath = recgli.noscreenshots[-nopath_index,]
dim(recgli.nopath) # 480 
nopath = temp
nopath[1,2] = sum(ifelse(duplicated(recgli.nopath$b_number)==FALSE & recgli.nopath$olddata==1, 1, 0))
nopath[1,3] = sum(ifelse(duplicated(recgli.nopath$t_number)==FALSE & recgli.nopath$olddata==1, 1, 0))
nopath[1,4] = sum(recgli.nopath$olddata==1)
nopath[2,2] = sum(ifelse(duplicated(recgli.nopath$b_number)==FALSE & recgli.nopath$newdata==1, 1, 0))
nopath[2,3] = sum(ifelse(duplicated(recgli.nopath$t_number)==FALSE & recgli.nopath$newdata==1, 1, 0))
nopath[2,4] = sum(recgli.nopath$newdata==1)
nopath[3,2:4]= as.numeric(nopath[1,2:4]) + as.numeric(nopath[2,2:4])
nopath

[1] 475  75

data,patients,scans,samples
old_po1,108,111,270
REC_HGG,69,71,205
TOTAL,177,182,475


In [23]:
write.csv(recgli.nopath, "5thParse_researchPath_withInVivo.csv", row.names = F)

In [24]:
recgli.noscreenshots[grep(5934, recgli.noscreenshots$t_number),]

b_number,t_number,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,roi.label,tumor_cell_evaluation,necrosis,f8_delicate,f8_simple,f8_complex,mib_1,bx_pure_treatment_effect,nfse,nfl,nt1c,nt1v,nt1d,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,phn_npar,recovn_npar,cni,ccri,crni,ncho,ncre,nnaa,laclip,nlip,nlac,olddata,newdata,comments,notes.,imaging_code,perf_quant,cbv_nlin,spec_quant,include_anat,include_diffu1000,include_diffu2000,include_diffu_all,include_perf,include_spec,include_one_advanced_mod,include_all_advanced_mods,X.CEL,X.NEL,X.NEC,sum,in_CEL,in_T2all,in_NEL,in_NEC,in_ROI,desired_hist,multnom_out,no_ffpe,waiting_on_path,rhgg_txe_analysis


## 8. Remove those whose imaging failed to be quantified for some reason 


In [25]:
recgli.nopath = read.csv("5thParse_researchPath_withInVivo.csv")
dim(recgli.nopath)

[1] 475  75

In [26]:
recgli.imagingfailed = recgli.nopath[-grep("biopsy_quant", recgli.nopath$imaging_code),]
dim(recgli.imagingfailed) # 438 
imagingfailed = temp
imagingfailed[1,2] = sum(ifelse(duplicated(recgli.imagingfailed$b_number)==FALSE & recgli.imagingfailed$olddata==1, 1, 0))
imagingfailed[1,3] = sum(ifelse(duplicated(recgli.imagingfailed$t_number)==FALSE & recgli.imagingfailed$olddata==1, 1, 0))
imagingfailed[1,4] = sum(recgli.imagingfailed$olddata==1)
imagingfailed[2,2] = sum(ifelse(duplicated(recgli.imagingfailed$b_number)==FALSE & recgli.imagingfailed$newdata==1, 1, 0))
imagingfailed[2,3] = sum(ifelse(duplicated(recgli.imagingfailed$t_number)==FALSE & recgli.imagingfailed$newdata==1, 1, 0))
imagingfailed[2,4] = sum(recgli.imagingfailed$newdata==1)
imagingfailed[3,2:4]= as.numeric(imagingfailed[1,2:4]) + as.numeric(imagingfailed[2,2:4])
imagingfailed

[1] 443  75

data,patients,scans,samples
old_po1,97,99,244
REC_HGG,69,71,199
TOTAL,166,170,443


In [27]:
write.csv(recgli.imagingfailed, "6thParse_researchPath_withInVivo.csv", row.names = F)

## 9. Remove ependymomas and other extraneous histologies


In [28]:
recgli.imagingfailed = read.csv("6thParse_researchPath_withInVivo.csv")

In [29]:
dim(recgli.imagingfailed)
colnames(recgli.imagingfailed)

[1] 443  75

[1] "b_number"                  "t_number"                 
 [3] "sf_number"                 "current_scan_date"        
 [5] "current_surgery_date"      "current_hist_grade"       
 [7] "current_hist_type"         "roi.label"                
 [9] "tumor_cell_evaluation"     "necrosis"                 
[11] "f8_delicate"               "f8_simple"                
[13] "f8_complex"                "mib_1"                    
[15] "bx_pure_treatment_effect"  "nfse"                     
[17] "nfl"                       "nt1c"                     
[19] "nt1v"                      "nt1d"                     
[21] "nadc.1"                    "nfa.1"                    
[23] "nev1.1"                    "nev2.1"                   
[25] "nev3.1"                    "nevrad.1"                 
[27] "nadc.2"                    "nfa.2"                    
[29] "nev1.2"                    "nev2.2"                   
[31] "nev3.2"                    "nevrad.2"                 
[33] "cbvn_nlin"                 "phn_nlin"                 
[35] "phn_npar"                  "recovn_npar"              
[37] "cni"                       "ccri"                     
[39] "crni"                      "ncho"                     
[41] "ncre"                      "nnaa"                     
[43] "laclip"                    "nlip"                     
[45] "nlac"                      "olddata"                  
[47] "newdata"                   "comments"                 
[49] "notes."                    "imaging_code"             
[51] "perf_quant"                "cbv_nlin"                 
[53] "spec_quant"                "include_anat"             
[55] "include_diffu1000"         "include_diffu2000"        
[57] "include_diffu_all"         "include_perf"             
[59] "include_spec"              "include_one_advanced_mod" 
[61] "include_all_advanced_mods" "X.CEL"                    
[63] "X.NEL"                     "X.NEC"                    
[65] "sum"                       "in_CEL"                   
[67] "in_T2all"                  "in_NEL"                   
[69] "in_NEC"                    "in_ROI"                   
[71] "desired_hist"              "multnom_out"              
[73] "no_ffpe"                   "waiting_on_path"          
[75] "rhgg_txe_analysis"

In [30]:
recgli.desiredhist = recgli.imagingfailed[recgli.imagingfailed$desired_hist==1,]
dim(recgli.desiredhist) 
desiredhist = temp
desiredhist[1,2] = sum(ifelse(duplicated(recgli.desiredhist$b_number)==FALSE & recgli.desiredhist$olddata==1, 1, 0))
desiredhist[1,3] = sum(ifelse(duplicated(recgli.desiredhist$t_number)==FALSE & recgli.desiredhist$olddata==1, 1, 0))
desiredhist[1,4] = sum(recgli.desiredhist$olddata==1)
desiredhist[2,2] = sum(ifelse(duplicated(recgli.desiredhist$b_number)==FALSE & recgli.desiredhist$newdata==1, 1, 0))
desiredhist[2,3] = sum(ifelse(duplicated(recgli.desiredhist$t_number)==FALSE & recgli.desiredhist$newdata==1, 1, 0))
desiredhist[2,4] = sum(recgli.desiredhist$newdata==1)
desiredhist[3,2:4]= as.numeric(desiredhist[1,2:4]) + as.numeric(desiredhist[2,2:4])
desiredhist

[1] 435  75

data,patients,scans,samples
old_po1,95,97,239
REC_HGG,68,70,196
TOTAL,163,167,435


In [31]:
write.csv(recgli.desiredhist, file = "7thParse_researchPath_withInVivo.csv", row.names = F)

## 10. Removing Necrotic samples

There are two definitions of necrosis: if the sample is in the NEC as defined by X.NEC, or if it's necrotic based on its pathology (with an "necrosis" score of 2) 

In [32]:
recgli.desiredhist = read.csv("7thParse_researchPath_withInVivo.csv")

In [33]:
dim(recgli.desiredhist)

[1] 435  75

In [34]:
desiredhist = temp
recgli.nonecrosis = recgli.desiredhist[-which(recgli.desiredhist$necrosis==2),]
recgli.nonecrosis = recgli.nonecrosis[-which(recgli.nonecrosis$in_NEC == 1),]
dim(recgli.nonecrosis) 
nonecrosis = temp
nonecrosis[1,2] = sum(ifelse(duplicated(recgli.nonecrosis$b_number)==FALSE & recgli.nonecrosis$olddata==1, 1, 0))
nonecrosis[1,3] = sum(ifelse(duplicated(recgli.nonecrosis$t_number)==FALSE & recgli.nonecrosis$olddata==1, 1, 0))
nonecrosis[1,4] = sum(recgli.nonecrosis$olddata==1)
nonecrosis[2,2] = sum(ifelse(duplicated(recgli.nonecrosis$b_number)==FALSE & recgli.nonecrosis$newdata==1, 1, 0))
nonecrosis[2,3] = sum(ifelse(duplicated(recgli.nonecrosis$t_number)==FALSE & recgli.nonecrosis$newdata==1, 1, 0))
nonecrosis[2,4] = sum(recgli.nonecrosis$newdata==1)
nonecrosis[3,2:4]= as.numeric(nonecrosis[1,2:4]) + as.numeric(nonecrosis[2,2:4])
nonecrosis

[1] 391  75

data,patients,scans,samples
old_po1,95,97,214
REC_HGG,63,65,177
TOTAL,158,162,391


In [35]:
write.csv(recgli.nonecrosis, file = "8thParse_researchPath_withInVivo.csv", row.names = F)

## 11. Only including rHGG or TxE designation: 

In [36]:
recgli.nonecrosis = read.csv("8thParse_researchPath_withInVivo.csv")

In [37]:
recgli.desiredpath = recgli.nonecrosis[recgli.nonecrosis$multnom_out=="TxE" |
                                         recgli.nonecrosis$multnom_out=="rHGG",]
dim(recgli.desiredpath)
table(recgli.desiredpath$multnom_out)

dim(recgli.desiredpath) # 438 
desiredpath = temp
desiredpath[1,2] = sum(ifelse(duplicated(recgli.desiredpath$b_number)==FALSE & recgli.desiredpath$olddata==1, 1, 0))
desiredpath[1,3] = sum(ifelse(duplicated(recgli.desiredpath$t_number)==FALSE & recgli.desiredpath$olddata==1, 1, 0))
desiredpath[1,4] = sum(recgli.desiredpath$olddata==1)
desiredpath[2,2] = sum(ifelse(duplicated(recgli.desiredpath$b_number)==FALSE & recgli.desiredpath$newdata==1, 1, 0))
desiredpath[2,3] = sum(ifelse(duplicated(recgli.desiredpath$t_number)==FALSE & recgli.desiredpath$newdata==1, 1, 0))
desiredpath[2,4] = sum(recgli.desiredpath$newdata==1)
desiredpath[3,2:4]= as.numeric(desiredpath[1,2:4]) + as.numeric(desiredpath[2,2:4])
desiredpath

[1] 322  75


IndDNI     PN   rHGG Ts1DNI    TxE 
     0      0    239      0     83 

[1] 322  75

data,patients,scans,samples
old_po1,85,87,171
REC_HGG,61,63,151
TOTAL,146,150,322


In [38]:
write.csv(recgli.desiredpath, file = "9thParse_researchPath_withInVivo.csv", row.names = F)